# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902047


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:05<01:18,  2.72s/it]

Rendering models:  10%|▉         | 3/31 [00:05<00:54,  1.96s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:51,  1.90s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:36,  1.39s/it]

...

Rendering models:  87%|████████▋ | 27/31 [00:10<00:00,  5.44it/s]

Rendering models:  90%|█████████ | 28/31 [00:10<00:00,  5.86it/s]

Rendering models:  94%|█████████▎| 29/31 [00:11<00:00,  4.52it/s]

Rendering models: 100%|██████████| 31/31 [00:11<00:00,  5.16it/s]

equidad1                              0.085822
mooberrysauce                         0.000810
ElisabethB                            0.010642
CDMP                                  0.045447
not-logged-in-621b62a5f3840545c538    0.418969
vosskftw                              0.022487
Thedunkmasta                          0.002665
OregonHiker                           0.000383
DrJackie                              0.000385
not-logged-in-eca205ebf9e395bd01ff    0.002980
Cadetrnd                              0.001065
not-logged-in-5f30b2bb9b7f9d49da41    0.002794
clairedeu                             0.001347
Susymuller                            0.002876
Lavadude                              0.053251
Paigery321                            0.003449
ShirleyL                              0.037071
saferguson                            0.000570
Anfrily                               0.001272
Jessica__                             0.010936
ktayl47                               0.000383
slambier     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())